In [2]:
pip install scikit-learn


In [1]:
from joblib import dump, load
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Naive Bayes Approach
from sklearn.naive_bayes import MultinomialNB
# Trees Approach
from sklearn.tree import DecisionTreeClassifier
# Ensemble Approach
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#KNN Approach
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#import the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#importing data
train_path = '/content/drive/MyDrive/data/train.csv'
test_path = '/content/drive/MyDrive/data/test.csv'

In [4]:
train_df = pd.read_csv(train_path)
train_df.head

<bound method NDFrame.head of       Loan_ID  Gender Married  ... Credit_History Property_Area Loan_Status
0    LP001002    Male      No  ...            1.0         Urban           Y
1    LP001003    Male     Yes  ...            1.0         Rural           N
2    LP001005    Male     Yes  ...            1.0         Urban           Y
3    LP001006    Male     Yes  ...            1.0         Urban           Y
4    LP001008    Male      No  ...            1.0         Urban           Y
..        ...     ...     ...  ...            ...           ...         ...
609  LP002978  Female      No  ...            1.0         Rural           Y
610  LP002979    Male     Yes  ...            1.0         Rural           Y
611  LP002983    Male     Yes  ...            1.0         Urban           Y
612  LP002984    Male     Yes  ...            1.0         Urban           Y
613  LP002990  Female      No  ...            0.0     Semiurban           N

[614 rows x 13 columns]>

In [5]:
#replace all 3+ with a random number greater than 3 in the Dependents column
import random

train_df.loc[(train_df.Dependents == "3+"), 'Dependents'] = random.randint(3,9)

In [6]:
#fill NaN with some values
train_df['Gender'] = train_df['Gender'].fillna('Male')
train_df['Married'] = train_df['Married'].fillna('No')
train_df['Dependents'] = train_df['Dependents'].fillna(2)
train_df['Education'] = train_df['Education'].fillna('Graduate')
train_df['Self_Employed'] = train_df['Self_Employed'].fillna('No')
train_df['ApplicantIncome'] = train_df['ApplicantIncome'].fillna(1560)
train_df['CoapplicantIncome'] = train_df['CoapplicantIncome'].fillna(0.0)
train_df['LoanAmount'] = train_df['LoanAmount'].fillna(100.0)
train_df['Loan_Amount_Term'] = train_df['Loan_Amount_Term'].fillna(360.0)
train_df['Credit_History'] = train_df['Credit_History'].fillna(1.0)
train_df['Property_Area'] = train_df['Property_Area'].fillna('Urban')
train_df['Loan_Status'] = train_df['Loan_Status'].fillna('Y')

In [7]:
train_df.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,45.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [8]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    object 
 2   Married            614 non-null    object 
 3   Dependents         614 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      614 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [9]:
#change some numeric columns to integers or float types
train_df.astype({'Dependents': 'int64','Loan_Amount_Term': 'int64','ApplicantIncome': 'float64'}).dtypes

Loan_ID               object
Gender                object
Married               object
Dependents             int64
Education             object
Self_Employed         object
ApplicantIncome      float64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term       int64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [10]:
#replacing categorical data with numbers
final_train_df = pd.get_dummies(data = train_df, columns =['Gender','Married','Education','Self_Employed','Property_Area'] )

In [11]:
final_train_df.head(20)

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,LP001002,0,5849,0.0,45.0,360.0,1.0,Y,0,1,1,0,1,0,1,0,0,0,1
1,LP001003,1,4583,1508.0,128.0,360.0,1.0,N,0,1,0,1,1,0,1,0,1,0,0
2,LP001005,0,3000,0.0,66.0,360.0,1.0,Y,0,1,0,1,1,0,0,1,0,0,1
3,LP001006,0,2583,2358.0,120.0,360.0,1.0,Y,0,1,0,1,0,1,1,0,0,0,1
4,LP001008,0,6000,0.0,141.0,360.0,1.0,Y,0,1,1,0,1,0,1,0,0,0,1
5,LP001011,2,5417,4196.0,267.0,360.0,1.0,Y,0,1,0,1,1,0,0,1,0,0,1
6,LP001013,0,2333,1516.0,95.0,360.0,1.0,Y,0,1,0,1,0,1,1,0,0,0,1
7,LP001014,3,3036,2504.0,158.0,360.0,0.0,N,0,1,0,1,1,0,1,0,0,1,0
8,LP001018,2,4006,1526.0,168.0,360.0,1.0,Y,0,1,0,1,1,0,1,0,0,0,1
9,LP001020,1,12841,10968.0,349.0,360.0,1.0,N,0,1,0,1,1,0,1,0,0,1,0


In [12]:
#the Loan_Status Column is our label so we will store it separately but let work on it
final_train_df.loc[(final_train_df.Loan_Status=='Y'), 'Loan_Status'] = 1 # replace Y with 1 
final_train_df.loc[(final_train_df.Loan_Status=='N'), 'Loan_Status'] = 0 # replace N with 0


In [13]:
final_train_df.astype({'Loan_Status': 'int64','Dependents': 'int64'}).dtypes

Loan_ID                     object
Dependents                   int64
ApplicantIncome              int64
CoapplicantIncome          float64
LoanAmount                 float64
Loan_Amount_Term           float64
Credit_History             float64
Loan_Status                  int64
Gender_Female                uint8
Gender_Male                  uint8
Married_No                   uint8
Married_Yes                  uint8
Education_Graduate           uint8
Education_Not Graduate       uint8
Self_Employed_No             uint8
Self_Employed_Yes            uint8
Property_Area_Rural          uint8
Property_Area_Semiurban      uint8
Property_Area_Urban          uint8
dtype: object

In [14]:
final_train_df.head(10)

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,LP001002,0,5849,0.0,45.0,360.0,1.0,1,0,1,1,0,1,0,1,0,0,0,1
1,LP001003,1,4583,1508.0,128.0,360.0,1.0,0,0,1,0,1,1,0,1,0,1,0,0
2,LP001005,0,3000,0.0,66.0,360.0,1.0,1,0,1,0,1,1,0,0,1,0,0,1
3,LP001006,0,2583,2358.0,120.0,360.0,1.0,1,0,1,0,1,0,1,1,0,0,0,1
4,LP001008,0,6000,0.0,141.0,360.0,1.0,1,0,1,1,0,1,0,1,0,0,0,1
5,LP001011,2,5417,4196.0,267.0,360.0,1.0,1,0,1,0,1,1,0,0,1,0,0,1
6,LP001013,0,2333,1516.0,95.0,360.0,1.0,1,0,1,0,1,0,1,1,0,0,0,1
7,LP001014,3,3036,2504.0,158.0,360.0,0.0,0,0,1,0,1,1,0,1,0,0,1,0
8,LP001018,2,4006,1526.0,168.0,360.0,1.0,1,0,1,0,1,1,0,1,0,0,0,1
9,LP001020,1,12841,10968.0,349.0,360.0,1.0,0,0,1,0,1,1,0,1,0,0,1,0


In [15]:
#creating data for ML
X = final_train_df.drop(['Loan_Status','Loan_ID'], axis = 1)
features_col = final_train_df.drop(['Loan_Status','Loan_ID'], axis = 1)
y = final_train_df['Loan_Status']
y=y.astype('int')

In [18]:
#splitting data into training and validation samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [21]:
#Training decision tree classifier
dt = DecisionTreeClassifier(criterion='entropy')
dt.fit(X_train,y_train)

# Trained Model Evaluation on Validation Dataset
confidence = dt.score(X_test, y_test)
# Validation Data Prediction
preds = dt.predict(X_test)
# Model Validation Accuracy
accuracy = accuracy_score(y_test, preds)
# Model Confusion Matrix
conf_mat = confusion_matrix(y_test, preds)
# Model Classification Report
clf_report = classification_report(y_test, preds)

print(confidence)
print('\n')
print(accuracy)
print('\n')
print(conf_mat)
print('\n')
print(clf_report)

0.7189189189189189


0.7189189189189189


[[ 32  32]
 [ 20 101]]


              precision    recall  f1-score   support

           0       0.62      0.50      0.55        64
           1       0.76      0.83      0.80       121

    accuracy                           0.72       185
   macro avg       0.69      0.67      0.67       185
weighted avg       0.71      0.72      0.71       185



In [22]:
#Training random forest classifier
rft = RandomForestClassifier(n_estimators=10)
rft.fit(X_train,y_train)

# Trained Model Evaluation on Validation Dataset
confidence = rft.score(X_test, y_test)
# Validation Data Prediction
preds = rft.predict(X_test)
# Model Validation Accuracy
accuracy = accuracy_score(y_test, preds)
# Model Confusion Matrix
conf_mat = confusion_matrix(y_test, preds)
# Model Classification Report
clf_report = classification_report(y_test, preds)

print(confidence)
print('\n')
print(accuracy)
print('\n')
print(conf_mat)
print('\n')
print(clf_report)

0.7567567567567568


0.7567567567567568


[[ 30  34]
 [ 11 110]]


              precision    recall  f1-score   support

           0       0.73      0.47      0.57        64
           1       0.76      0.91      0.83       121

    accuracy                           0.76       185
   macro avg       0.75      0.69      0.70       185
weighted avg       0.75      0.76      0.74       185



In [23]:
#Training KNN classifier
knn = KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)
knn.fit(X_train,y_train)

# Trained Model Evaluation on Validation Dataset
confidence = knn.score(X_test, y_test)
# Validation Data Prediction
preds = knn.predict(X_test)
# Model Validation Accuracy
accuracy = accuracy_score(y_test, preds)
# Model Confusion Matrix
conf_mat = confusion_matrix(y_test, preds)
# Model Classification Report
clf_report = classification_report(y_test, preds)

print(confidence)
print('\n')
print(accuracy)
print('\n')
print(conf_mat)
print('\n')
print(clf_report)

0.6162162162162163


0.6162162162162163


[[ 10  54]
 [ 17 104]]


              precision    recall  f1-score   support

           0       0.37      0.16      0.22        64
           1       0.66      0.86      0.75       121

    accuracy                           0.62       185
   macro avg       0.51      0.51      0.48       185
weighted avg       0.56      0.62      0.56       185



In [65]:
myInput = []
for i in range(0, len(cols_feat)):
    myInput.append(0)
print(myInput)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [66]:
input_vals = []
count = 12
for i in range(0,11):
  vals =input('Enter : ')
  count = count - 1
  print(count,' more need')
  #store symtoms in a list
  input_vals.append(vals)

Enter : 0
11  more need
Enter : 5849
10  more need
Enter : 0
9  more need
Enter : 45
8  more need
Enter : 360
7  more need
Enter : 1
6  more need
Enter : Male
5  more need
Enter : No
4  more need
Enter : Graduate
3  more need
Enter : No
2  more need
Enter : Urban
1  more need


In [17]:
cols_feat = list(features_col)
print(cols_feat)

['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Gender_Female', 'Gender_Male', 'Married_No', 'Married_Yes', 'Education_Graduate', 'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes', 'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban']


In [67]:
print(input_vals)

['0', '5849', '0', '45', '360', '1', 'Male', 'No', 'Graduate', 'No', 'Urban']


In [68]:
my_im_dict = {}
k = 0
for i in input_vals:
  my_im_dict[k]= i
  k+=1

In [69]:
print(my_im_dict)

{0: '0', 1: '5849', 2: '0', 3: '45', 4: '360', 5: '1', 6: 'Male', 7: 'No', 8: 'Graduate', 9: 'No', 10: 'Urban'}


In [70]:
# preprocessing user input for the machine model
for i in range(0,len(myInput)):
    if i < 6:
      myInput[i] = my_im_dict[i]

    elif i == 6:
      if my_im_dict[i] == "Female": myInput[i] = 1
      else: myInput[i+1] = 1
      

    elif i == 7:
      if my_im_dict[i] == "No": myInput[i+1] = 1
      else: myInput[i+2] = 1
      
      

    elif i == 8:
      if my_im_dict[i] == "Graduate": myInput[i+2] = 1
      else: myInput[i+3] = 1
      
      

    elif i == 9:
      if my_im_dict[i] == "No": myInput[i+3] = 1
      else: myInput[i+4] = 1
      
      

    elif i == 10:
      
      if my_im_dict[i] == "Rural": myInput[i+4] = 1
      elif my_im_dict[i] == "Semiurban": myInput[i+5] = 1
      else: myInput[i+6] = 1
  

In [71]:
print(myInput)

['0', '5849', '0', '45', '360', '1', 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1]


In [72]:
# Making single prediction Prediction
input_mat = [myInput]
predict = rft.predict(input_mat)
print(predict)

[1]


In [86]:
#preprocessing test data
test_df = pd.read_csv(test_path)
test_df.head

<bound method NDFrame.head of       Loan_ID Gender Married  ... Loan_Amount_Term Credit_History Property_Area
0    LP001015   Male     Yes  ...            360.0            1.0         Urban
1    LP001022   Male     Yes  ...            360.0            1.0         Urban
2    LP001031   Male     Yes  ...            360.0            1.0         Urban
3    LP001035   Male     Yes  ...            360.0            NaN         Urban
4    LP001051   Male      No  ...            360.0            1.0         Urban
..        ...    ...     ...  ...              ...            ...           ...
362  LP002971   Male     Yes  ...            360.0            1.0         Urban
363  LP002975   Male     Yes  ...            360.0            1.0         Urban
364  LP002980   Male      No  ...            360.0            NaN     Semiurban
365  LP002986   Male     Yes  ...            360.0            1.0         Rural
366  LP002989   Male      No  ...            180.0            1.0         Rural

[367 rows

In [87]:
#replace all 3+ with a random number greater than 3 in the Dependents column
import random

test_df.loc[(test_df.Dependents == "3+"), 'Dependents'] = random.randint(3,9)

In [88]:
#fill NaN with some values
test_df['Gender'] = test_df['Gender'].fillna('Male')
test_df['Married'] = test_df['Married'].fillna('No')
test_df['Dependents'] = test_df['Dependents'].fillna(2)
test_df['Education'] = test_df['Education'].fillna('Graduate')
test_df['Self_Employed'] = test_df['Self_Employed'].fillna('No')
test_df['ApplicantIncome'] = test_df['ApplicantIncome'].fillna(1560)
test_df['CoapplicantIncome'] = test_df['CoapplicantIncome'].fillna(0.0)
test_df['LoanAmount'] = test_df['LoanAmount'].fillna(100.0)
test_df['Loan_Amount_Term'] = test_df['Loan_Amount_Term'].fillna(360.0)
test_df['Credit_History'] = test_df['Credit_History'].fillna(1.0)
test_df['Property_Area'] = test_df['Property_Area'].fillna('Urban')


In [89]:
#change some numeric columns to integers or float types
test_df.astype({'Dependents': 'int64','Loan_Amount_Term': 'int64','ApplicantIncome': 'float64'}).dtypes

Loan_ID               object
Gender                object
Married               object
Dependents             int64
Education             object
Self_Employed         object
ApplicantIncome      float64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term       int64
Credit_History       float64
Property_Area         object
dtype: object

In [90]:
#replacing categorical data with numbers
final_test_df = pd.get_dummies(data = test_df, columns =['Gender','Married','Education','Self_Employed','Property_Area'] )

In [91]:
final_test_df.head(20)

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,LP001015,0,5720,0,110.0,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
1,LP001022,1,3076,1500,126.0,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
2,LP001031,2,5000,1800,208.0,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
3,LP001035,2,2340,2546,100.0,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
4,LP001051,0,3276,0,78.0,360.0,1.0,0,1,1,0,0,1,1,0,0,0,1
5,LP001054,0,2165,3422,152.0,360.0,1.0,0,1,0,1,0,1,0,1,0,0,1
6,LP001055,1,2226,0,59.0,360.0,1.0,1,0,1,0,0,1,1,0,0,1,0
7,LP001056,2,3881,0,147.0,360.0,0.0,0,1,0,1,0,1,1,0,1,0,0
8,LP001059,2,13633,0,280.0,240.0,1.0,0,1,0,1,1,0,1,0,0,0,1
9,LP001067,0,2400,2400,123.0,360.0,1.0,0,1,1,0,0,1,1,0,0,1,0


In [94]:
final_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Loan_ID                  367 non-null    object 
 1   Dependents               367 non-null    object 
 2   ApplicantIncome          367 non-null    int64  
 3   CoapplicantIncome        367 non-null    int64  
 4   LoanAmount               367 non-null    float64
 5   Loan_Amount_Term         367 non-null    float64
 6   Credit_History           367 non-null    float64
 7   Gender_Female            367 non-null    uint8  
 8   Gender_Male              367 non-null    uint8  
 9   Married_No               367 non-null    uint8  
 10  Married_Yes              367 non-null    uint8  
 11  Education_Graduate       367 non-null    uint8  
 12  Education_Not Graduate   367 non-null    uint8  
 13  Self_Employed_No         367 non-null    uint8  
 14  Self_Employed_Yes        3

In [96]:
final_test_df.astype({'Dependents': 'int64'}).dtypes

Loan_ID                     object
Dependents                   int64
ApplicantIncome              int64
CoapplicantIncome            int64
LoanAmount                 float64
Loan_Amount_Term           float64
Credit_History             float64
Gender_Female                uint8
Gender_Male                  uint8
Married_No                   uint8
Married_Yes                  uint8
Education_Graduate           uint8
Education_Not Graduate       uint8
Self_Employed_No             uint8
Self_Employed_Yes            uint8
Property_Area_Rural          uint8
Property_Area_Semiurban      uint8
Property_Area_Urban          uint8
dtype: object

In [97]:
#creating data for ML
test_features = final_test_df.drop(['Loan_ID'], axis = 1)


In [98]:
#testing the model
dt_predit = dt.predict(test_features)

In [99]:
rft_predict = rft.predict(test_features)

In [100]:
print(dt_predit)
print('\n')
print(rft_predict)

[1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0
 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0
 1 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 1 0
 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 1 0
 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1
 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1 1 1 1 1 0
 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1]


[1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0